In [1]:
import sys
from scipy.sparse import dok_matrix
from numpy.random import choice


In [5]:
f = open("FILES_TO_READ.txt","r")
file_names = f.read()
file_names = file_names.split()

corpus = ""

for file_name in file_names:
    with open("data/"+file_name, 'r') as f:
            corpus+=f.read()
corpus = corpus.replace('\n',' ')
corpus = corpus.replace('\t',' ')
corpus = corpus.replace('“', ' " ')
corpus = corpus.replace('”', ' " ')
for spaced in ['.','-',',','!','?','(','—',')']:
    corpus = corpus.replace(spaced, ' {0} '.format(spaced))

print("Corpus length: ",len(corpus),"characters")

Corpus length:  33924882 characters


In [4]:
corpus_words = corpus.split(' ')
corpus_words= [word for word in corpus_words if word != '']
print("Words in corpus: ",len(corpus_words))

Words in corpus:  6918692


In [6]:
distinct_words = list(set(corpus_words))
word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
distinct_words_count = len(list(set(corpus_words)))
print("Distinct words in corpus: ",distinct_words_count)

Distinct words in corpus:  59014


<hr>

In [7]:
k = 3
sets_of_k_words = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]

from scipy.sparse import dok_matrix

sets_count = len(list(set(sets_of_k_words)))
next_after_k_words_matrix = dok_matrix((sets_count, len(distinct_words)))

distinct_sets_of_k_words = list(set(sets_of_k_words))
k_words_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_k_words)}

for i, word in enumerate(sets_of_k_words[:-k]):

    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+k]]
    next_after_k_words_matrix[word_sequence_idx, next_word_idx] +=1

In [9]:
def sample_next_word_after_sequence(word_sequence, alpha):
    next_word_vector = next_after_k_words_matrix[k_words_idx_dict[word_sequence]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()
    return choice(distinct_words, 1, p=likelihoods.toarray()[0])[0]
    #return weighted_choice(distinct_words, likelihoods.toarray())

def stochastic_chain(seed, chain_length=15, seed_length=2,alpha=0):
    current_words = seed.split(' ')
    if len(current_words) != seed_length:
        print(len(current_words))
        print(seed_length)
        raise ValueError(f'wrong number of words, expected {seed_length}')
    sentence = seed

    for _ in range(chain_length):
        sentence+=' '
        next_word = sample_next_word_after_sequence(' '.join(current_words),alpha)
        sentence+=next_word
        current_words = current_words[1:]+[next_word]
    return sentence


In [10]:
NLINE_ON = ["MATT:","MARISHA:","LAURA:","SAM:","ASHLEY:","TALIESIN:","TRAVIS:","LIAM:","ORION:"]
NOSPACE_ON = [".",",","'","\"",":",";","?","!","-"]

def format_out(string_out):
    str_formatted = ""
    for word in string_out.split():
        if word in NLINE_ON:
            str_formatted += "\n\n" + word
        elif word in NOSPACE_ON:
            str_formatted += word
        else:
            str_formatted += " " + word
    return str_formatted

In [12]:
seed = "Caleb is a"
length = 45

print(format_out(stochastic_chain(seed, chain_length=length,seed_length=len(seed.split()),alpha=0)))

 Caleb is a speed reader it's no big deal.

LIAM: Oh, you haven't seen in a while?

LIAM: We need to get the gold?

LAURA: I feel really rusty.

LAURA: How much is it?

MARISHA: Yeah!

LAURA: Wah- wah.
